In [8]:
# Imports necessary libraries for Agents.
import os
from dotenv import load_dotenv
from openai import AzureOpenAI
from azure.storage.blob import BlobServiceClient
from azure.identity import DefaultAzureCredential, get_bearer_token_provider

load_dotenv()

# Ensure your client is initialized first
def initialize_client():
    # Get environment variables for endpoint and deployment
    endpoint = os.getenv("AZURE_AI_PROJECT_ENDPOINT")
    deployment = os.getenv("DEPLOYMENT_NAME")

    # Initialize Azure OpenAI client with Entra ID authentication  
    cognitiveServicesResource = os.getenv('AZURE_COGNITIVE_SERVICES_RESOURCE', 'YOUR_COGNITIVE_SERVICES_RESOURCE')  
    token_provider = get_bearer_token_provider(  
        DefaultAzureCredential(),  
        f'{cognitiveServicesResource}.default',
    )  

    # Create AzureOpenAI client
    client = AzureOpenAI(  
        azure_endpoint=endpoint,  
        api_key=os.getenv("AZURE_OPENAI_KEY"),
        api_version=os.getenv("AZURE_OPENAI_VERSION")
    )
    return client, deployment

# Initialize the client before creating the PlannerAgent instance
client, deployment = initialize_client()

In [4]:
# Search Agent 

class SearchAgent:
    """
    A class to represent the search agent.
    """
    def search_agent(self, output_citation_title, test_file_answers_name):
        """
        Lists test-file-container contents to user to receive input. Input is used to fetch a user selection
        for the 'test-file' they wish to send to agent 2 for grading.
        """
        # Creates string to retrieve test information of selected test from OpenAI client object.
        content_string = f"Extracts all the answers from the 'Part B: Student Answers' section of the test in the \"{output_citation_title}.pdf\" files."

        # Creates second chat object to retrieve the test information of the test chosen by user.
        completion2 = client.chat.completions.create(  
            model=deployment,  
            messages=[
        {
                "role": "system",
                "content": "You are a precision answer extraction system. Follow these protocols:\n\n **File Selection**:\n   - Process ONLY files matching: `test-file-*-submitted.pdf`."
            },
            {
                "role": "user",
                "content": content_string
            },
            {
                "role": "assistant",
                "content": "The requested information is not available in the retrieved data. Please try another query or topic."
            },
            {
                "role": "user",
                "content": content_string
            }   
        ],  
            max_tokens=800,  
            temperature=0.0,  
            top_p=0.95,  
            frequency_penalty=0,  
            presence_penalty=0,  
            stop=None,  
            extra_body={  
                "data_sources": [  
                    {  
                        "type": "azure_search",  
                        "parameters": {  
                            "endpoint": os.getenv("AZURE_SEARCH_ENDPOINT"),  
                            "index_name": os.getenv("AZURE_SEARCH_INDEX"),  
                            "authentication": {  
                                "type": "api_key",
                                "key": os.getenv("AZURE_SEARCH_KEY")
                            }  
                        }  
                    }  
                ]  
            }  
        )


        response_data = completion2.model_dump()
        test_info = "STUDENT ANSWERS: \n" + completion2.choices[0].message.content

        # Connects to storage blob service.
        blob_service_client = BlobServiceClient.from_connection_string(
            os.getenv("STORAGE_BLOB_CONNECTION_STRING")
        )

        # Creates variables containing the name of the storage container and the name of the file being written.
        container_name = "test-file-container"  
        blob_name = test_file_answers_name

        # Creates the blob object.
        blob_client = blob_service_client.get_blob_client(
            container=container_name,
            blob=blob_name
        )
        # Uploads the new file to the storage blob container.
        blob_client.upload_blob(test_info, overwrite=True)
        # Prints message for successful upload.
        print(f"File '{blob_name}' uploaded to container '{container_name}'.")
        return blob_name

In [6]:

# Grading Agent
class GradingAgent:
    """
    Class to represent the functionality of the grading agent. 
    """
    def __init__(self, client, deployment):
        self.client = client
        self.deployment = deployment
        
    # Creates chat object to retrieve student grades and the answer key grades.
    def get_student_answers(self, blob_name):
        student_answer_client = self.client.chat.completions.create(  
           model=self.deployment,
           messages = [
                {
                    "role": "system",
                    "content": """You are a precise answer extraction tool. Follow these rules:
                        
                        1. EXTRACT ANSWERS:
                           - Find exactly 3 answers
                           - Answers must appear in order (Question 1 to 3)
                           - Capture only the selected option for each
                        
                        2. REQUIRED FORMAT:
                           1. Selected: [value]
                           2. Selected: [value]
                           3. Selected: [value]
                        
                        3. HANDLING DIFFERENT FORMATS:
                           - For multiple choice: Extract the letter (A/B/C/D)
                           - For yes/no: Extract "Yes" or "No"
                           - For written answers: Extract first 3 words max
                           - If blank/unanswered: Mark as "Blank"
                        
                        4. VALIDATION:
                           - If any answer is missing/unclear: return "ERROR: Incomplete answers"
                           - If format differs: return "ERROR: Unexpected answer format"
                    """
                },
                {
                    "role": "user",
                    "content": f"""From {blob_name}, extract Part B answers with 100% accuracy:
            
                    - Document may contain:
                      • Checkboxes (☒/☐)
                      • Letters in parentheses (A)
                      • Highlighted text
                      • Handwritten answers
                    
                    Return ONLY in this exact format:
                    1. Selected: [value]
                    2. Selected: [value]
                    3. Selected: [value]"""
                }
        ],  
            max_tokens=100,  
            temperature=0.0,  
            top_p=0.95,  
            frequency_penalty=0,  
            presence_penalty=0,  
            stop=None,  
            extra_body={  
                "data_sources": [  
                    {  
                        "type": "azure_search",  
                        "parameters": {  
                            "endpoint": os.getenv("AZURE_SEARCH_ENDPOINT"),  
                            "index_name": os.getenv("AZURE_SEARCH_INDEX"),  
                            "authentication": {  
                                "type": "api_key",
                                "key": os.getenv("AZURE_SEARCH_KEY")
                            }  
                        }  
                    }  
                ]  
            }  
          )
        response_data = student_answer_client.model_dump()
        student_answer_info = student_answer_client.choices[0].message.content
        return(student_answer_info)

    def get_answer_key(self, blob_name):
        answer_key_name = f"{blob_name[:-21]}answers.pdf"
        answer_key_client = client.chat.completions.create(  
            model=deployment,  
            messages = [
              {
                  "role": "system",
                  "content": """You extract exactly 3 correct answers from a plain-text answer key. Follow these rules:
                                1. Start after the line that says "Answers:"
                                2. FORMAT each as: "1. Selected: X" (X = A/B/C/D, Yes/No, or first 3 words)
                                3. NORMALIZE:
                                   - A/B/C/D → UPPERCASE
                                   - Yes/No → Capitalized
                                   - Short text answers → lowercase
                                    4. If unclear or missing: return "Blank"
                                    5. Output EXACTLY 3 LINES, nothing else
                                """
                },
                {
                  "role": "user",
                  "content": f"Extract correct answers from the following answer key:{answer_key_name}"
                }
        ],
            max_tokens=100,  
            temperature=0.0,  
            top_p=1,  
            frequency_penalty=0,  
            presence_penalty=0,  
            stop=None,  
            extra_body={  
                 "data_sources": [  
                      {  
                       "type": "azure_search",  
                        "parameters": {  
                            "endpoint": os.getenv("AZURE_SEARCH_ENDPOINT"),  
                            "index_name": os.getenv("AZURE_SEARCH_INDEX"),  
                            "authentication": {  
                                "type": "api_key",
                                "key": os.getenv("AZURE_SEARCH_KEY")
                            }  
                        }  
                    }  
                 ]  
             }  
         )
        response_data = answer_key_client.model_dump()
        correct_answer_info = answer_key_client.choices[0].message.content
        return correct_answer_info

    def grade_answers(self, blob_name, student_answer_info, grade_answers):

        submitted_answers = self.get_student_answers(blob_name)
        correct_answers = self.get_answer_key(blob_name)
        
        # Instructions to retrieve grade information.
        test_grade = f"""
            ACT AS A PRECISE ANSWER GRADING SYSTEM
            
            === INSTRUCTIONS ===
            1. NORMALIZE BOTH ANSWER SETS:
               - Remove ALL prefixes ("Selected:", "Answer:", "Option")
               - Trim whitespace
               - Convert to uppercase
            
            2. COMPARE NORMALIZED ANSWERS:
               - Must be EXACT matches after normalization
               - Example: "B" matches "B" but not " B" or "b"
            
            3. CALCULATE RESULTS:
               - Grade = (Correct Answers / Total Questions) * 100
               - List incorrect question numbers
            
            === RAW ANSWER DATA ===
            SUBMITTED ANSWERS:
            {submitted_answers}
            
            CORRECT ANSWERS:
            {correct_answers}
            
            === REQUIRED OUTPUT FORMAT ===
            Total Questions: [number]
            Correct Answers: [number]
            Incorrect Questions: [list]
            Grade Percentage: [number]%
        """
        grading_client = client.chat.completions.create(  
            model=deployment,  
            messages=[
             {
                "role": "system", 
                "content": "You are an exact answer grading system. Follow the instructions precisely."
            },
            {
                "role": "user",
                "content": test_grade
            }
        ],  
            max_tokens=800,  
            temperature=0.0,  
            top_p=1,  
            frequency_penalty=0,  
            presence_penalty=0,  
            stop=None,  
        )
        response_data = grading_client.model_dump()
        response_output = grading_client.choices[0].message.content
        return response_output
        

In [8]:
# Report Agent
class ReportAgent:
    """
    Class to represent the functionality of the report agent. Utilizes chat agent to write a brief report
    based on grade data generated in the grading agent.
    """
    def __init__(self, client, deployment):
        self.client = client
        self.deployment = deployment

    """
    Method within the report agent that creates the GPT agent which generates the report. 
    """
    def report_generation(self, response_output):
        # Content string for retrieving student answers file.
        report_generation = f"""
        Generate a brief report based on the grade information in {response_output}. In the report, write the grade as a percentage. Keep 
        report token length under 400.
        """
        
        # Creates chat object to retrieve student grades and the answer key grades.
        report_writer = client.chat.completions.create(  
            model=deployment,  
             messages=[
            {
                "role": "system",
                "content": f"You are a writing assistant. Write a report based on the information generated in {response_output}."
            },
            {
                "role": "user",
                "content": report_generation
            }
        ],  
            # Sets GPT settings for report generation.
            max_tokens=400,  
            temperature=0.7,  
            top_p=0.95,  
            frequency_penalty=0,  
            presence_penalty=0,  
            stop=None,  
        )
        response_data = report_writer.model_dump()
        report_info = report_writer.choices[0].message.content
        print("""
_____________________________________________________________________________________________________________________________________________\n""" + report_info)



In [ ]:
## Planner Agent
"""
The Agent that orchestrates the other agents to work together.
"""
class PlannerAgent:
    """
    Method that initializes the class.
    """
    def __init__(self, client, deployment):
        self.client = client           
        self.deployment = deployment
    """
    Main method for the PlannerAgent class. Calls other agents to retrieve information required
    to generate report based on answers submitted by the user from the storage container. Retrieves user
    input for test-file selection with input validation. Retrieves user input based on list presented by
    the user_selection_client, which utilizes the search service to fetch test-file names for files tagged with
    'submitted' in the filename from the storage blob container. Once input is received and validated by application, agent calls Search
    agent to insert a newly created file with ONLY the submitted answers from the file selected into the storage container. Once complete, the
    Grading agent is called to grade the answers submitted based on information in the file named 'test-file-*-answers.pdf'. Once completed, the
    Report agent is called to generate a report based on the grade given by the Grading agent and print the report to user.
    """
    def main(self):
                
        # Creates user_selection GPT client to retrieve the test file names from the container for selection.
        user_selection_client = client.chat.completions.create(  
            model=deployment,  
            # Prompt which retrieves file name information from storage blob container.
            messages=[
            {
                "role": "system",
                "content": """
                    You are a precise file retrieval system. Follow these rules EXACTLY:
                    
                    1. FILE PATTERN: Only return files matching EXACTLY: 'test-file-*-submitted.pdf'
                    2. FORMAT: Return each matching file on a new line with its exact name
                    3. EXAMPLES:
                       - GOOD: test-file-1-submitted.pdf
                       - BAD: test-file-1-answers.pdf
                    4. COMPLETENESS: You MUST return ALL matching files in the index
                   """
            },
            {
                "role": "user",
                "content": "List each file with the name which follows the following format: \"test-file-*-submitted.pdf\""
            },
            {
                "role": "assistant",
                "content": "I will display each file with the name format of \"test-file-*-submitted.pdf\""
            },
            {
                "role": "user",
                "content": "List each file with the name which follows the following format: \"test-file-*-submitted.pdf\n For example: 'test-file-{integer}-submitted.pdf' List the files in descending order by the test-file number at x in 'test-file-x-submitted.pdf'."
            }
        ],  
            max_tokens=800,  
            temperature=0.7,  
            top_p=0.95,  
            frequency_penalty=0,  
            presence_penalty=0,  
            stop=None,  
            # Utilizes Azure Search to locate files within the Storage Blob Container.
            extra_body={  
                "data_sources": [  
                    {  
                       "type": "azure_search",  
                        "parameters": {  
                            "endpoint": os.getenv("AZURE_SEARCH_ENDPOINT"),  
                            "index_name": os.getenv("AZURE_SEARCH_INDEX"),  
                            "authentication": {  
                                "type": "api_key",
                                "key": os.getenv("AZURE_SEARCH_KEY")
                            }  
                        }  
                    }  
                ]  
            }  
        )
        
        # Creates list to hold container file citation data.
        response_data = user_selection_client.model_dump()
        citations = response_data['choices'][0]['message']['context']['citations']

        # Prints message to user to retrieve input for file selection.
        print("Please select one of the following files to create a report on: ")
        count = 0
        citation_title_list = []
        # Inserts each file title into 
        for citation in citations:
            count += 1
            print(f"{count} : {citation['title']}")
            citation_title_list.append(citation['title'])
        # Validates user input for file selection.
        while True:
            user_input = input("Please enter a number (listed on the left) to select a file: ")
    
            # Validate input is a number
            if not user_input.isdigit():
                print("Invalid input. Please enter a number.")
                continue
    
            num = int(user_input)

            # Validate number is in range
            if num > 0 and num <= len(citations):
                break  # Valid input, exit loop
            else:
                print(f"Invalid selection. Please enter a number between 1 and {len(citations)}.")

        # Continue with your code using the valid 'num' variable
        print(f"You selected option: {num}")

        # Creates answer sheet file name
        output_citation_title = citation_title_list[int(user_input)-1][:-4]
        test_file_answers_name = f"{output_citation_title}-answers.txt"

        # Calls first agent to send the newly created file with. 
        blob_name = SearchAgent.search_agent(self, output_citation_title, test_file_answers_name)
        # Initialize GradingAgent with required dependencies
        grading_agent = GradingAgent(self.client, self.deployment)
        
        # Get answers
        student_answers = grading_agent.get_student_answers(blob_name)
        answer_key = grading_agent.get_answer_key(blob_name)
        
        # Grade answers
        grading_result = grading_agent.grade_answers(blob_name, student_answers, answer_key)
        # Prints report
        ReportAgent.report_generation(self, grading_result)
# Creates an instance of PlannerAgent.
agent = PlannerAgent(client, deployment)
# Calls the main method of PlannerAgent.
selected_file = agent.main()